### Read chat.db into a csv

In [ ]:
import sqlite3
import pandas as pd

# Connect to the chat.db file
conn = sqlite3.connect('../data/chat.db')

# Execute a query to fetch the data
cursor = conn.cursor()
cursor.execute('SELECT * FROM messages')

# Fetch all the rows
rows = cursor.fetchall()

# Create a DataFrame from the fetched rows
df = pd.DataFrame(rows)

# Save the DataFrame as a CSV file
df.to_csv('../data/chat.csv', index=False)
